```
This notebook sets up and runs a Cartesian analog to the Galewsky
2004 test case, defined in the paper 
"An initial-value problem for testing numerical models of the global
shallow-water equations"

Copyright (C) 2016, 2017, 2018  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
import scipy as scipy
from scipy import integrate
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import os
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import datetime
import sys

from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
#rc('figure', figsize=(16.0, 12.0))
#rc('animation', html='html5')
plt.rcParams["animation.html"] = "jshtml"

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#%setup_logging --out galewsky.log
%cuda_context_handler gpu_ctx

# Galewsky 2004 test case

The test case is defined for spheric global shallow water models representing atmospheric flow, in the paper 
```
Joseph Galewsky, Richard K. Scott & Lorenzo M. Polvani (2004) An initial-
value problem for testing numerical models of the global shallow-water equations, Tellus A:
Dynamic Meteorology and Oceanography, 56:5, 429-440, DOI: 10.3402/tellusa.v56i5.14436
```
which can be found here: https://doi.org/10.3402/tellusa.v56i5.14436

The case describes a smooth yet moving around the globe initialized with a balanced $\eta$-field which describes a steady-state solution.
A perturbation is then added to the initial conditions, which generates turbulent solutions for the inviscid case, and a well-defined reference solution for the viscid case.

The **first goal** is therefore to rewrite the initial conditions to Cartesian coordinates, and reproduce the steady state solution. We here consider whether we should use the $\beta$-model, or if it is sufficient to consider the case with a constant Coriolis force.

The **second goal** is to add the perturbation and see if we can recognize the reference solution provided by Galewsky. It will be interesting to see whether CTCS is able to obtain this solution.

In [ ]:
# DEFINE PARAMETERS

sim_args = {
    "gpu_ctx": gpu_ctx,
    "nx": 500, "ny": 200,
    "dx": 10000.0, "dy": 10000.0,
    "dt": 20,
    "g": 9.81,
    "f": 1.2e-4,
    "coriolis_beta": 0.0,
    "r": 0.0,
    "boundary_conditions": Common.BoundaryConditions(1,2,1,2)
}

cdklm_args = {
    "rk_order": 2,
}

depth = 100
rossby_radius = np.sqrt(sim_args["g"]*depth)/sim_args["f"]
phase_speed = np.sqrt(sim_args['g']*depth)
periode = sim_args['dx']*sim_args['nx']/phase_speed

print("rossby_radius: ", rossby_radius)
print("phase_speed:   ", phase_speed)
print("periode:       ", periode)

In [ ]:
def add_hovmuller(i, t, hovmuller_data, data):
    hovmuller_data['t'][i] = t
    for y, hovmuller in hovmuller_data['hovmuller']:
        hovmuller[i,:] = data[y,:]

def sim_animation(simulator, T, sub_dt=10, hovmuller_data=None):
    """
    Creates an animation of the simulator based on T frames, with frames sub_dt*simulator.dt appart.
    
    If hovmuller_data is provided, cross sections of the simulation is stored in the hovmuller_data arrays.
    hovmuller_data is expected to be of the format:
    hovmuller_data = {'t': 1D array of size T which will contain the simulation times,
            'hovmuller': [(y1, 2D array of size T times nx to store the cross sections),
                          (y2, 2D array of size T times nx to store the cross sections),
                           ...]}
    """
    eta1, u1, v1 = sim.download(interior_domain_only=True)
    
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(16, 16))
    domain_extent = [0, sim.nx*sim.dx, 0, sim.ny*sim.dy]
    
    eta_max = eta1.max()*0.5
    hu_max = max(v1.max(), u1.max())
    
    ax_eta = plt.subplot(3,1,1)
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='bottom', vmin=-eta_max, vmax=eta_max, extent=domain_extent)
    #ax_eta.colorbar()
    
    ax_u = plt.subplot(3,1,2)
    sp_u = ax_u.imshow(u1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    #ax_u.colorbar()
    
    ax_v = plt.subplot(3,1,3)
    sp_v = ax_v.imshow(v1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    #ax_v.colorbar()
    
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim.step(sub_dt*sim.dt)
        else:
            t = 0.0
        eta1, u1, v1 = sim.download(interior_domain_only=True)
        
        if hovmuller_data is not None:
            add_hovmuller(i, t, hovmuller_data, eta1)
        
        return
        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)
        
        fig.sca(ax_u)
        sp_u.set_data(u1)
        
        fig.sca(ax_v)
        sp_v.set_data(v1)
        
        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        print(".", end='')

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    return anim

In [ ]:
def init_eta(eta, ghost_cells):
    ny, nx = eta.shape
    x_0 = nx/2
    y_0 = ghost_cells[2]
    for j in range(0, ny):
        for i in range(0, nx):
            r_j = np.sqrt((j-y_0)**2)*sim_args["dy"]
            r_i = np.sqrt((i-x_0)**2)*sim_args["dx"]
            
            f_func = 1.0 + np.tanh((-r_i + rossby_radius)/(rossby_radius/3))
            #f_func = 1.0 + np.tanh((-r_i + rossby_radius)/(rossby_radius/2))
        
            eta[j,i] = np.exp(-r_j/rossby_radius)*f_func

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])
dataShapeHi = (sim_args["ny"] + ghosts[0]+ghosts[2]+1, 
             sim_args["nx"] + ghosts[1]+ghosts[3]+1)

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hv  = np.zeros(dataShape, dtype=np.float32, order='C');
hu  = np.zeros(dataShape, dtype=np.float32, order='C');

center_x = (sim_args["nx"]+2)/2
center_y = 0


for j in range(0, sim_args["ny"]+4):
    for i in range(0,sim_args["nx"]+4):
        scale = 1
        #r_i = bump_size*(i-center_x)**2 *sim_args["dx"]**2
        #r_j = bump_size*(j-center_y) *sim_args["dy"]
        
        r = np.sqrt((i-center_x)**2 + (j-center_y)**2)*sim_args["dx"]
        
        #f_func = np.exp(-r_i/rossby_radius)
        
        #eta[j,i] = np.exp(-scale*r/rossby_radius)
        #hu[ j,i] = (sim_args["g"]*depth/(sim_args["f"]*rossby_radius))*f_func
        

init_eta(eta, ghosts)

        
const = sim_args["g"]*depth/sim_args["f"]
for j in range(2,sim_args["ny"]+2):
    for i in range(2,sim_args["nx"]+2):
        #hv[j,i] =  const*(eta[j, i+1] - eta[j,i-1])/(2*sim_args["dx"])
        hu[j,i] = -const*(eta[j+1, i] - eta[j-1,i])/(2*sim_args["dy"])
        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])

                             
cdklm_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : np.ones(dataShapeHi, dtype=np.float32, order='C')*depth
}


sim = CDKLM16.CDKLM16(**sim_args, **cdklm_init_args)

T = 101
sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes/
#sub_dt = periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes/
CDKLM_hovmuller = {'t': np.zeros(T),
                   'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                 (10, np.zeros((T, sim_args["nx"]))), 
                                 (50, np.zeros((T, sim_args["nx"])))]}

sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=CDKLM_hovmuller)
#sim_animation(sim, 1)

In [ ]:
#CTCS
ghosts = [1,1,1,1]
validDomain = np.array([1,1,1,1]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hu = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
hv = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  

center_x = (sim_args["nx"]+2)/2
center_y = 0
y_mid = (sim_args["ny"]+2)/2

H = np.ones(eta.shape, dtype=np.float32, order='C')*depth


for j in range(0, sim_args["ny"]+2):
    for i in range(0,sim_args["nx"]+2):
        scale = 1/3.0
        
        r_j = np.sqrt((j-center_y)**2)*sim_args["dy"]
        r_i = np.sqrt((i-center_x)**2)*sim_args["dx"]
        
        
        f_func = np.exp(-scale*r_i/rossby_radius)
        
        eta[j,i] = np.exp(-r_j/rossby_radius)*f_func
        #eta[j,i] = f_func
        

            

for j in range(0, sim_args["ny"]+2):
    for i in range(0,sim_args["nx"]+2):
        scale = 0.3
        
        r_j = np.sqrt((j-center_y)**2)*sim_args["dy"]
        
        
        f_func = 1.0 + np.tanh((-np.sqrt((i-center_x)**2)*sim_args["dx"] + rossby_radius)/(rossby_radius/3))
        
        #eta[j,i] = np.exp(-r_j/rossby_radius)*f_func
        


init_eta(eta, ghosts)


for j in range(1,sim_args["ny"]+1):
    for i in range(1,sim_args["nx"]+2):
        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])

#
#plt.plot(eta[2:,250])
#plt.plot(eta[2,:])
#plt.plot(eta[20,:])
#osidhgd

        
ctcs_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : H,
    "A" : 0
}


sim = CTCS.CTCS(**sim_args, **ctcs_init_args)

T = 101
sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes
CTCS_hovmuller = {'t': np.zeros(T),
                  'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                (10, np.zeros((T, sim_args["nx"]))), 
                                (50, np.zeros((T, sim_args["nx"])))]}

sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=CTCS_hovmuller)
#sim_animation(sim, 1)

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])
dataShapeHi = (sim_args["ny"] + ghosts[0]+ghosts[2]+1, 
             sim_args["nx"] + ghosts[1]+ghosts[3]+1)

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hv  = np.zeros(dataShape, dtype=np.float32, order='C');
hu  = np.zeros(dataShape, dtype=np.float32, order='C');

center_x = (sim_args["nx"]+2)/2
center_y = 0


for j in range(0, sim_args["ny"]+4):
    for i in range(0,sim_args["nx"]+4):
        scale = 1
        #r_i = bump_size*(i-center_x)**2 *sim_args["dx"]**2
        #r_j = bump_size*(j-center_y) *sim_args["dy"]
        
        r = np.sqrt((i-center_x)**2 + (j-center_y)**2)*sim_args["dx"]
        
        #f_func = np.exp(-r_i/rossby_radius)
        
        #eta[j,i] = np.exp(-scale*r/rossby_radius)
        #hu[ j,i] = (sim_args["g"]*depth/(sim_args["f"]*rossby_radius))*f_func
        

init_eta(eta, ghosts)

        
const = sim_args["g"]*depth/sim_args["f"]
for j in range(2,sim_args["ny"]+2):
    for i in range(2,sim_args["nx"]+2):
        #hv[j,i] =  const*(eta[j, i+1] - eta[j,i-1])/(2*sim_args["dx"])
        hu[j,i] = -const*(eta[j+1, i] - eta[j-1,i])/(2*sim_args["dy"])
        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])

                             
kp07_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : np.ones(dataShapeHi, dtype=np.float32, order='C')*depth
}


sim = KP07.KP07(**sim_args, **cdklm_init_args)

T = 101
sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes/
#sub_dt = periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes/
KP07_hovmuller = {'t': np.zeros(T),
                   'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                 (10, np.zeros((T, sim_args["nx"]))), 
                                 (50, np.zeros((T, sim_args["nx"])))]}

sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=KP07_hovmuller)
#sim_animation(sim, 1)

In [ ]:
#FBL
ghosts = [1,1,1,1]
validDomain = np.array([1,1,1,1]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hu = np.zeros((dataShape[0], dataShape[1]-1), dtype=np.float32);
hv = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  

center_x = (sim_args["nx"]+2)/2
center_y = 0
y_mid = (sim_args["ny"]+2)/2

H = np.ones(eta.shape, dtype=np.float32, order='C')*depth

init_eta(eta, ghosts)


for j in range(sim_args["ny"]):
    for i in range(sim_args["nx"]+1):
        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])

#
#plt.plot(eta[2:,250])
#plt.plot(eta[2,:])
#plt.plot(eta[20,:])
#osidhgd

        
fbl_init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : H
}

reload(FBL)
sim = FBL.FBL(**sim_args, **fbl_init_args)

T = 101
sub_dt = 10*periode/(sim_args['dt']*(T-1)) # So that we run the wave for two whole periodes
FBL_hovmuller = {'t': np.zeros(T),
                  'hovmuller': [(0,  np.zeros((T, sim_args["nx"]))), 
                                (10, np.zeros((T, sim_args["nx"]))), 
                                (50, np.zeros((T, sim_args["nx"])))]}

sim_animation(sim, T, sub_dt=sub_dt, hovmuller_data=FBL_hovmuller)
#sim_animation(sim, 1)

In [ ]:
def plot_hovmuller(hovmuller_data, title):
    domain_extent = [0, sim_args['nx']*sim_args['dx'], 0, hovmuller_data['t'][-1]]
    max_eta = eta.max()
    for hovmuller in hovmuller_data['hovmuller']:

        fig = plt.figure()
        plt.imshow(hovmuller[1], 
                   origin='lower', extent=domain_extent, aspect='auto',
                   interpolation='none', vmin=0, vmax=max_eta,)
        plt.title(title +  " for y = " + str(hovmuller[0]))
        plt.colorbar()
        
plot_hovmuller(CTCS_hovmuller, "CTCS")
plot_hovmuller(CDKLM_hovmuller, "CDKLM")
plot_hovmuller(KP07_hovmuller, "KP07")


In [ ]:
# Check solution after one periode:
def plot_periodic_solution(hovmuller_data, title):
    fig = plt.figure(figsize=(15,5))
    for hovmuller in hovmuller_data['hovmuller']:
        plt.plot(hovmuller[1][0 , 180:320], 'b:')
        plt.plot(hovmuller[1][10, 180:320], 'y-.')
        plt.plot(hovmuller[1][50, 180:320], 'r--')
        plt.plot(hovmuller[1][-1, 180:320], 'g-')
        plt.title(title)
    plt.grid()
    print('final t/10: ', hovmuller_data['t'][-1]/10)
    print('half  t/5:  ', hovmuller_data['t'][50]/5)
    print('one p t:    ', hovmuller_data['t'][10])
    print('periode:    ', periode)
    print('caption: blue = initial data, red = 5 periods, green = 10 periods')

plot_periodic_solution(CTCS_hovmuller, "CTCS")
plot_periodic_solution(FBL_hovmuller, "FBL")
print('A: ', ctcs_init_args['A'])
plot_periodic_solution(CDKLM_hovmuller, "CDKLM")
plot_periodic_solution(KP07_hovmuller, "KP07")


In [ ]:
T = 30
hovmuller_south = np.zeros((T, sim_args["nx"]))
hovmuller_10 = np.zeros((T, sim_args["nx"]))
hovmuller_50 = np.zeros((T, sim_args["nx"]))
hovmuller_t = np.zeros(T)

CTCS_hovmuller = {'t': hovmuller_t,
                  'hovmuller': [(0, hovmuller_south), (10, hovmuller_10), (50, hovmuller_50)]}

def test_hovmuller(T, hovmuller_data, data):
    for t in range(T):
        add_hovmuller(t, t*100, hovmuller_data, data)
        #hovmuller_data['t'][t] = t*100
        #for y, hovmuller in hovmuller_data['hovmuller']:
        #    hovmuller[t,:] = data[y,:]
        #    #print(y, hovmuller[t])

def add_hovmuller(i, t, hovmuller_data, data):
    hovmuller_data['t'][i] = t
    for y, hovmuller in hovmuller_data['hovmuller']:
        hovmuller[i,:] = data[y,:]
        
test_hovmuller(T, CTCS_hovmuller, eta[1:-1, 1:-1])
            
#print(hovmuller_south.transpose())
#print(hovmuller_10.transpose())
#print(hovmuller_50.transpose())
plt.imshow(hovmuller_south)
plt.colorbar()
fig = plt.figure()
plt.imshow(hovmuller_10)
plt.colorbar()
fig = plt.figure()
plt.imshow(hovmuller_50)
plt.colorbar()
print(hovmuller_t)



Beskrivelse av init + boundary conditions. Beskrivelse av valg av parametere sammenlignet med Rossbyradien.

Analytisk  fasehastighet, og plot som viser at den stemmer. HovmÃ¼ller diagram med karakteristikkene.

Vurdere Ã¥ si noe om energibetraktninger.
Sjekke at bÃ¸lgen er lik etter en "runde" (en periode). Fasehastighet er $\sqrt{gH}$.


In [ ]:
plt.plot(eta[2:,250])
plt.plot(eta[2,:])
plt.plot(eta[20,:])
eta2, hu2, hv2 = sim.download()
plt.plot(eta2[2,:])
plt.plot(eta2[20,:])

#plt.imshow(hv2, origin='lower')
#plt.colorbar()

fig=plt.figure(figsize=(15,15))
plt.imshow(hu2, origin='lower')
plt.colorbar()

fig=plt.figure(figsize=(15,15))
plt.imshow(eta, origin='lower')
plt.colorbar()

## Investigating the initial conditions based on spheric coordinates